## Query structure mirrors pif structure
* You can query subsystems, processing steps, properties, conditions of properties etc. by creating a query that matches the object hierarchy to the section you want to query


In [1]:
from citrination_client import CitrinationClient
from citrination_client import PifSystemReturningQuery, PifSystemQuery, FieldQuery, ValueQuery
from citrination_client import PropertyQuery, DataQuery, DatasetQuery, ChemicalFieldQuery, ChemicalFilter, Filter

from os import environ
from pypif import pif

client = CitrinationClient(environ['CITRINATION_API_KEY'], 'https://citrination.com')

## Flattening the PIF structure

extract_as creates a flattened dictionary structure mapping user supplied keys to objects in the PIF that match within the query

extract_all is an option for extract_as that pulls a list of all objects at the level in the hierarchy that match the query

Let's search for the "Enthalpy of Formation" property:

In [2]:
dataset_id = 150675
query_size = 10

query = PifSystemReturningQuery(
            size=query_size,
            query=DataQuery(
                dataset=DatasetQuery(
                    id=[Filter(equal=str(dataset_id))]
                ),
                chemical_formula=ChemicalFieldQuery(
                    extract_as='formula',
                    filter=ChemicalFilter(
                            equal='CdTe')),
                system=PifSystemQuery(
                    properties=PropertyQuery(
                        extract_all=True,
                        name=FieldQuery(
                            filter=[Filter(equal="Enthalpy of Formation")]),
                        value=FieldQuery(
                            extract_as="formation_enthalpy",
                            extract_all=True)
                    )
                )
            ))

query_result = client.search(query)
print( "{} hits\n".format(query_result.total_num_hits))
print( "Extracted fields:")
for i in range(2):
    print( pif.dumps(query_result.hits[i].extracted, indent=2))

69640 hits

Extracted fields:
{
  "formation_enthalpy": [
    "0.0"
  ]
}
{
  "formation_enthalpy": [
    "0.1074050600000005"
  ]
}


## Chemical formula search
Citrine has developed specialized search functionality specifically for chemical formulas. The analyser parses the search string and recognizes chemical entities such as elements and stoichiometries to find chemically relevant results.

1. You can use a `simple_chemical_search` with a simple search string like "PbSe" or,
2. You can structure a `PifSystemReturningQuery` with more detailed elemental and stoichiometric strings.

In [3]:
mp_dataset_id = 150675

search_result = client.simple_chemical_search(chemical_formula="PbSe", include_datasets=[mp_dataset_id])
print( "{} hits \n".format(search_result.total_num_hits))

for i in range(search_result.total_num_hits):
    print( pif.dumps(search_result.hits[i].extracted))
    print("\n")

4 hits 

{"property_units": "g/cm$^3$", "name": "Lead selenide - HP", "property_value": "8.767862185821011", "chemical_formula": "PbSe", "property_name": "Density"}


{"property_units": "g/cm$^3$", "name": "Lead selenide", "property_value": "4.062929739915243", "chemical_formula": "PbSe", "property_name": "Density"}


{"property_units": "g/cm$^3$", "name": "Clausthalite", "property_value": "7.872521935843158", "chemical_formula": "PbSe", "property_name": "Density"}


{"property_units": "g/cm$^3$", "name": "Nickel lead selenide (3/2/2)", "property_value": "9.207453481307569", "chemical_formula": "Ni3(PbSe)2", "property_name": "Density"}




In [4]:
mp_dataset_id = 150675

query = PifSystemReturningQuery(
            size=5,
            random_results=True,
            query=DataQuery(
                dataset=DatasetQuery(
                    id=[Filter(equal=str(mp_dataset_id))]
                ),
                system=PifSystemQuery(
                    chemical_formula=ChemicalFieldQuery(
                        extract_all=True,
                        extract_as='formula'),
                    )
                )
            )

search_result = client.search(query)
print( "{} hits".format(search_result.total_num_hits))
for i in range(5):
    print( pif.dumps(search_result.hits[i].extracted))
    

query = PifSystemReturningQuery(
            size=5,
            random_results=True,
            query=DataQuery(
                dataset=DatasetQuery(
                    id=[Filter(equal=str(mp_dataset_id))]
                ),
                system=PifSystemQuery(
                    chemical_formula=ChemicalFieldQuery(
                        extract_all=True,
                        extract_as='formula',
                        filter=ChemicalFilter(
                            equal='Ga')
                    )
                )
            )
        )

search_result = client.search(query)
print( "{} hits".format(search_result.total_num_hits))
for i in range(5):
    print( pif.dumps(search_result.hits[i].extracted))


query = PifSystemReturningQuery(
            size=5,
            random_results=True,
            query=DataQuery(
                dataset=DatasetQuery(
                    id=[Filter(equal=str(mp_dataset_id))]
                ),
                system=PifSystemQuery(
                    chemical_formula=ChemicalFieldQuery(
                        extract_all=True,
                        extract_as='formula',
                        filter=ChemicalFilter(
                            equal='?x?y?z')
                    )
                )
            )
        )

search_result = client.search(query)
print( "{} hits".format(search_result.total_num_hits))
for i in range(5):
    print( pif.dumps(search_result.hits[i].extracted))


query = PifSystemReturningQuery(
            size=5,
            random_results=True,
            query=DataQuery(
                dataset=DatasetQuery(
                    id=[Filter(equal=str(mp_dataset_id))]
                ),
                system=PifSystemQuery(
                    chemical_formula=ChemicalFieldQuery(
                        extract_all=True,
                        extract_as='formula',
                        filter=ChemicalFilter(
                            equal='?xOy')
                    )
                )
            )
        )

search_result = client.search(query)
print( "{} hits".format(search_result.total_num_hits))
for i in range(5):
    print( pif.dumps(search_result.hits[i].extracted))
    

query = PifSystemReturningQuery(
            size=5,
            random_results=True,
            query=DataQuery(
                dataset=DatasetQuery(
                    id=[Filter(equal=str(mp_dataset_id))]
                ),
                system=PifSystemQuery(
                    chemical_formula=ChemicalFieldQuery(
                        extract_all=True,
                        extract_as='formula',
                        filter=ChemicalFilter(
                            equal='?1O1')
                    )
                )
            )
        )

search_result = client.search(query)
print( "{} hits".format(search_result.total_num_hits))
for i in range(5):
    print( pif.dumps(search_result.hits[i].extracted))

69640 hits
{"formula": "Fe5O3F7"}
{"formula": "Tb2BaTe4"}
{"formula": "PrRuO3"}
{"formula": "Li3TiMn3O8"}
{"formula": "WF5"}
23 hits
{"formula": "Ga(IO3)3"}
{"formula": "Ga(SbBr)4"}
{"formula": "Ga(BiBr)4"}
{"formula": "Ga(MoSe2)4"}
{"formula": "Ga"}
33017 hits
{"formula": "EuTl2Pd"}
{"formula": "YVO3"}
{"formula": "Er3SnC"}
{"formula": "SbAsO3"}
{"formula": "Pr10S14O"}
1577 hits
{"formula": "LiO3"}
{"formula": "Sm2O"}
{"formula": "BiO2"}
{"formula": "VO2"}
{"formula": "CrO2"}
121 hits
{"formula": "TiO"}
{"formula": "MgO"}
{"formula": "CuO"}
{"formula": "FeO"}
{"formula": "FeO"}


## Logical operations

SHOULD, MUST, OPTIONAL, MUST_NOT

In [5]:
query = PifSystemReturningQuery(
            size=5,
            random_results=True,
            query=DataQuery(
                dataset=DatasetQuery(
                    id=[Filter(equal=str(mp_dataset_id))]
                ),
                system=PifSystemQuery(
                    chemical_formula=[
                        ChemicalFieldQuery(
                            extract_as='formula',
                            filter=ChemicalFilter(
                                equal='?1O1'),
                            logic="MUST_NOT"),
                        ChemicalFieldQuery(
                        extract_as='formula',
                            filter=ChemicalFilter(
                                equal='?xOy')
                        )]
                )
            )
        )

search_result = client.search(query)
print( "{} hits\n".format(search_result.total_num_hits))
for i in range(5):
    print( pif.dumps(search_result.hits[i].extracted))


    
query = PifSystemReturningQuery(
            size=5,
            random_results=True,
            query=DataQuery(
                dataset=DatasetQuery(
                    id=[Filter(equal=str(mp_dataset_id))]
                ),
                system=PifSystemQuery(
                    chemical_formula=ChemicalFieldQuery(
                        extract_as='formula'
                    ),
                    properties=[
                        PropertyQuery(
                            name=FieldQuery(
                                filter=[Filter(equal="Enthalpy of Formation")]),
                            value=FieldQuery(
                                extract_as="H_f",
                                logic="MUST")
                        ),
                        PropertyQuery(
                            name=FieldQuery(
                                filter=[Filter(equal="Band Gap")]),
                            value=FieldQuery(
                                filter=[Filter(min=1E-5)],
                                extract_as="bandgap",
                                logic="MUST")
                        ),
                         PropertyQuery(
                            name=FieldQuery(
                                filter=[Filter(equal="Poisson Ratio")]),
                            value=FieldQuery(
                                extract_as="Poisson Ratio",
                                logic="MUST")
                        )]
                )
            )
        )

search_result = client.search(query)
print( "{} hits\n".format(search_result.total_num_hits))
for i in range(5):
    print( pif.dumps(search_result.hits[i].extracted, indent=2))

1456 hits

{"formula": "SiO2"}
{"formula": "PtO3"}
{"formula": "LaO3"}
{"formula": "SiO2"}
{"formula": "Rb2O"}
69640 hits

{
  "bandgap": "3.7474999999999996",
  "formula": "Ca10P6SeO24",
  "H_f": "-3.2900484016381757"
}
{
  "formula": "RbNaO3",
  "H_f": "-0.5273712257500002"
}
{
  "bandgap": "1.8445",
  "formula": "Ce8Nd2O19",
  "H_f": "-3.890026235301723"
}
{
  "bandgap": "2.6097000000000006",
  "formula": "Dy2Zr2O7",
  "H_f": "-3.885406168977273"
}
{
  "formula": "Ce(GePt)2",
  "H_f": "-0.9025403209999994"
}
